In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [61]:
import pandas as pd

df_combined = pd.read_csv('/content/drive/MyDrive/RAISE25/combined_dataset_with_entities.csv')

df_combined.head()

,date,title,source,number_of_characters_title,number_of_words_title,day_of_week,month,year,quarter,is_weekend,category,language,cleaned_title,extracted_entities,filtered_entities,cleaned_entities,final_entities
0,2024-10-14,"will.i.am and Fyilicia on the AI revolution, i...",English,122,20,Monday,October,2024,4,False,Other,NaN,william and fyilicia on the ai revolution incl...,"[('will', 'ORG', 0.7079097), ('i', 'ORG', 0.51...","[('will', 'ORG', 0.708), ('i', 'ORG', 0.513), ...","[('will', 'ORG', 0.7079097), ('i', 'ORG', 0.51...","[('will', 'ORG', 0.7079097), ('i', 'ORG', 0.51..."
1,2024-02-21,Intel Launches World’s First Systems Foundry D...,English,117,18,Wednesday,February,2024,1,False,Other,NaN,intel launches worlds first systems foundry de...,"[('Intel Launch', 'ORG', 0.90908504), ('World'...","[('Intel Launch', 'ORG', 0.909), ('World', 'OR...","[('Intel Launch', 'ORG', 0.90908504), ('World'...","[('Intel Launch', 'ORG', 0.90908504), ('World'..."
2,2024-02-05,The Unique Challenges of Selling Enterprise AI,English,54,9,Monday,February,2024,1,False,Career,NaN,the unique challenges of selling enterprise ai,"[('Enterprise AI', 'MISC', 0.58336604)]",[],"[('Enterprise AI', 'MISC', 0.58336604)]","[('Enterprise AI', 'MISC', 0.58336604)]"
3,2024-08-28,Contentious California AI bill passes legislat...,English,88,11,Wednesday,August,2024,3,False,Other,NaN,contentious california ai bill passes legislat...,"[('California AI', 'MISC', 0.7799265)]",[],"[('California AI', 'MISC', 0.7799265)]","[('California AI', 'MISC', 0.7799265)]"
4,2024-10-15,"Exploring Genius, Creation, and Humanity in th...",English,82,14,Tuesday,October,2024,4,False,Other,NaN,exploring genius creation and humanity in the ...,"[('Age of AI', 'MISC', 0.94894797)]",[],"[('Age of AI', 'MISC', 0.94894797)]","[('Age of AI', 'MISC', 0.94894797)]"


In [62]:
import ast

def safe_eval(entities):
    try:
        return ast.literal_eval(entities)
    except:
        return []  # In case parsing fails, return an empty list

df_combined["final_entities"] = df_combined["final_entities"].apply(safe_eval)


In [63]:
df_combined["final_entities"].iloc[0]

[('will', 'ORG', 0.7079097),
 ('i', 'ORG', 0.51297027),
 ('The Voice UK', 'ORG', 0.9171531),
 ('Tech & Science Daily', 'ORG', 0.99780107)]

In [64]:
for entity in df_combined["final_entities"].iloc[0]:
    print(entity[0])  # Entity name, type, confidence

will
i
The Voice UK
Tech & Science Daily


In [67]:
from transformers import pipeline
import ast
import pandas as pd

# Load FinBERT (financial/business sentiment model)
finbert_model = pipeline("sentiment-analysis", model="ProsusAI/finbert")

# ✅ Function to analyze sentiment with FinBERT and return decimal sentiment
def analyze_sentiment_finbert(text):
    result = finbert_model(text)[0]
    label = result["label"]
    score = result["score"]

    if label == "negative":
        return -score
    elif label == "neutral":
        return 0.0
    return score  # Positive stays positive

# ✅ Function to assign sentiment to entities in each headline
def extract_entity_sentiment_finbert(row):
    sentiment_score = analyze_sentiment_finbert(row["title"])
    entities = row["final_entities"]

    # Pair each entity with the sentiment from the headline
    entity_sentiments = [(entity[0], sentiment_score, row["title"]) for entity in entities]

    return entity_sentiments

# ✅ Flatten Entity-Sentiment Pairs Across All Headlines
entity_sentiment_pairs = []

for _, row in df_combined.iterrows():
    entity_sentiments = extract_entity_sentiment_finbert(row)
    entity_sentiment_pairs.extend(entity_sentiments)

df_combined["entity_sentiment_per_article"] = df_combined.apply(extract_entity_sentiment_finbert, axis=1)

Device set to use cuda:0


In [75]:
df_combined.head()

,date,title,source,number_of_characters_title,number_of_words_title,day_of_week,month,year,quarter,is_weekend,category,language,cleaned_title,extracted_entities,filtered_entities,cleaned_entities,final_entities,entity_sentiment_per_article
0,2024-10-14,"will.i.am and Fyilicia on the AI revolution, i...",English,122,20,Monday,October,2024,4,False,Other,NaN,william and fyilicia on the ai revolution incl...,"[('will', 'ORG', 0.7079097), ('i', 'ORG', 0.51...","[('will', 'ORG', 0.708), ('i', 'ORG', 0.513), ...","[('will', 'ORG', 0.7079097), ('i', 'ORG', 0.51...","[(will, ORG, 0.7079097), (i, ORG, 0.51297027),...","[(will, 0.0, will.i.am and Fyilicia on the AI ..."
1,2024-02-21,Intel Launches World’s First Systems Foundry D...,English,117,18,Wednesday,February,2024,1,False,Other,NaN,intel launches worlds first systems foundry de...,"[('Intel Launch', 'ORG', 0.90908504), ('World'...","[('Intel Launch', 'ORG', 0.909), ('World', 'OR...","[('Intel Launch', 'ORG', 0.90908504), ('World'...","[(Intel Launch, ORG, 0.90908504), (World, ORG,...","[(Intel Launch, 0.0, Intel Launches World’s Fi..."
2,2024-02-05,The Unique Challenges of Selling Enterprise AI,English,54,9,Monday,February,2024,1,False,Career,NaN,the unique challenges of selling enterprise ai,"[('Enterprise AI', 'MISC', 0.58336604)]",[],"[('Enterprise AI', 'MISC', 0.58336604)]","[(Enterprise AI, MISC, 0.58336604)]","[(Enterprise AI, 0.0, The Unique Challenges of..."
3,2024-08-28,Contentious California AI bill passes legislat...,English,88,11,Wednesday,August,2024,3,False,Other,NaN,contentious california ai bill passes legislat...,"[('California AI', 'MISC', 0.7799265)]",[],"[('California AI', 'MISC', 0.7799265)]","[(California AI, MISC, 0.7799265)]","[(California AI, 0.0, Contentious California A..."
4,2024-10-15,"Exploring Genius, Creation, and Humanity in th...",English,82,14,Tuesday,October,2024,4,False,Other,NaN,exploring genius creation and humanity in the ...,"[('Age of AI', 'MISC', 0.94894797)]",[],"[('Age of AI', 'MISC', 0.94894797)]","[(Age of AI, MISC, 0.94894797)]","[(Age of AI, 0.0, Exploring Genius, Creation, ..."


In [54]:
headline_to_find = "Study: AI could lead to inconsistent outcomes in home surveillance"

df_combined[df_combined["title"] == headline_to_find]

,date,title,source,number_of_characters_title,number_of_words_title,day_of_week,month,year,quarter,is_weekend,category,language,cleaned_title,extracted_entities,filtered_entities,cleaned_entities,final_entities,entity_sentiment_per_article
18,2024-09-19,Study: AI could lead to inconsistent outcomes ...,English,77,13,Thursday,September,2024,3,False,Education,NaN,study ai could lead to inconsistent outcomes i...,[],[],[],"[(Artificial Intelligence, MISC, 1.0)]","[(Artificial Intelligence, -0.9667720794677734..."


In [76]:
def contains_negative_sentiment(entity_sentiments):
    if not entity_sentiments:  # Handle empty lists
        return False
    return any(entity[1] < 0 for entity in entity_sentiments)

# Filter rows where any entity has negative sentiment
negative_sentiment_rows = df_combined[df_combined["entity_sentiment_per_article"].apply(contains_negative_sentiment)]

# Show the first few rows
negative_sentiment_rows.head()


,date,title,source,number_of_characters_title,number_of_words_title,day_of_week,month,year,quarter,is_weekend,category,language,cleaned_title,extracted_entities,filtered_entities,cleaned_entities,final_entities,entity_sentiment_per_article
18,2024-09-19,Study: AI could lead to inconsistent outcomes ...,English,77,13,Thursday,September,2024,3,False,Education,NaN,study ai could lead to inconsistent outcomes i...,[],[],[],"[(Artificial Intelligence, MISC, 1.0)]","[(Artificial Intelligence, -0.9667720794677734..."
20,2024-08-20,AI chatbot blamed for psychosocial workplace t...,English,88,14,Tuesday,August,2024,3,False,Career,NaN,ai chatbot blamed for psychosocial workplace t...,"[('Bunbury', 'LOC', 0.99611557)]","[('B', 'LOC', 0.996), ('##unbury', 'LOC', 0.952)]","[('Bunbury', 'LOC', 0.99611557)]","[(Bunbury, LOC, 0.99611557)]","[(Bunbury, -0.866581380367279, AI chatbot blam..."
21,2024-10-08,AI is fixing — and ruining — our photos,English,51,12,Tuesday,October,2024,4,False,Other,NaN,ai is fixing and ruining our photos,[],[],[],"[(Artificial Intelligence, MISC, 1.0)]","[(Artificial Intelligence, -0.8278920650482178..."
26,2024-09-19,UK Bank Reveals 28% Of Adults Have Fallen Vict...,English,150,30,Thursday,September,2024,3,False,Other,NaN,uk bank reveals 28 of adults have fallen victi...,"[('UK Bank', 'ORG', 0.9977975)]","[('UK Bank', 'ORG', 0.998)]","[('UK Bank', 'ORG', 0.9977975)]","[(UK Bank, ORG, 0.9977975)]","[(UK Bank, -0.8717716336250305, UK Bank Reveal..."
30,2024-09-30,California governor vetoes bill to create firs...,English,94,13,Monday,September,2024,3,False,Other,NaN,california governor vetoes bill to create firs...,"[('California', 'LOC', 0.99976057)]","[('California', 'LOC', 1.0)]","[('California', 'LOC', 0.99976057)]","[(California, LOC, 0.99976057)]","[(California, -0.7241971492767334, California ..."


In [79]:
df_combined.head()

,date,title,source,number_of_characters_title,number_of_words_title,day_of_week,month,year,quarter,is_weekend,category,language,cleaned_title,extracted_entities,filtered_entities,cleaned_entities,final_entities,entity_sentiment_per_article
0,2024-10-14,"will.i.am and Fyilicia on the AI revolution, i...",English,122,20,Monday,October,2024,4,False,Other,NaN,william and fyilicia on the ai revolution incl...,"[('will', 'ORG', 0.7079097), ('i', 'ORG', 0.51...","[('will', 'ORG', 0.708), ('i', 'ORG', 0.513), ...","[('will', 'ORG', 0.7079097), ('i', 'ORG', 0.51...","[(will, ORG, 0.7079097), (i, ORG, 0.51297027),...","[(will, 0.0, will.i.am and Fyilicia on the AI ..."
1,2024-02-21,Intel Launches World’s First Systems Foundry D...,English,117,18,Wednesday,February,2024,1,False,Other,NaN,intel launches worlds first systems foundry de...,"[('Intel Launch', 'ORG', 0.90908504), ('World'...","[('Intel Launch', 'ORG', 0.909), ('World', 'OR...","[('Intel Launch', 'ORG', 0.90908504), ('World'...","[(Intel Launch, ORG, 0.90908504), (World, ORG,...","[(Intel Launch, 0.0, Intel Launches World’s Fi..."
2,2024-02-05,The Unique Challenges of Selling Enterprise AI,English,54,9,Monday,February,2024,1,False,Career,NaN,the unique challenges of selling enterprise ai,"[('Enterprise AI', 'MISC', 0.58336604)]",[],"[('Enterprise AI', 'MISC', 0.58336604)]","[(Enterprise AI, MISC, 0.58336604)]","[(Enterprise AI, 0.0, The Unique Challenges of..."
3,2024-08-28,Contentious California AI bill passes legislat...,English,88,11,Wednesday,August,2024,3,False,Other,NaN,contentious california ai bill passes legislat...,"[('California AI', 'MISC', 0.7799265)]",[],"[('California AI', 'MISC', 0.7799265)]","[(California AI, MISC, 0.7799265)]","[(California AI, 0.0, Contentious California A..."
4,2024-10-15,"Exploring Genius, Creation, and Humanity in th...",English,82,14,Tuesday,October,2024,4,False,Other,NaN,exploring genius creation and humanity in the ...,"[('Age of AI', 'MISC', 0.94894797)]",[],"[('Age of AI', 'MISC', 0.94894797)]","[(Age of AI, MISC, 0.94894797)]","[(Age of AI, 0.0, Exploring Genius, Creation, ..."


In [78]:
df_combined.to_csv('/content/drive/MyDrive/RAISE25/df_combined_with_entity_sentiment.csv', index=False)
print("✅ DataFrame saved to your Google Drive as df_combined_with_entity_sentiment.csv")


✅ DataFrame saved to your Google Drive as df_combined_with_entity_sentiment.csv
